# Домашнее задание к лекции «Понятие класса»

### Задание 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса www.cbr-xml-daily.ru...ly_json.js

In [48]:
import requests

class FindCurrent:
    def __init__(self, format_='value'):
        self.format_ = format_
        self.name = 'Name'
        self.value = 'Value'
    
    def get_current(self):
        self.db = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.db.json()
    
    def snow_name(self):
        return self.name

all_base = FindCurrent(format_='full')                            # Присвоение переменной для использования класса
db = all_base.get_current()                                       # Выгрузка данных с сайта через класс

val_base = dict(db['Valute'])                                     # Отделение сутевого содержимого из общей выгруженной инф-ции
# print(val_base)                                                 # Контрольный вывод формата содержимого

result_name = ''                                                  # Объявление и обнуление переменной для результата "Название"
result_value = 0.0                                                # Объявление и обнуление переменной для результата "Курс"

for curr in val_base:                                             # Цикл обработки каждой из выгруженных валют (подряд)
    line = {}                                                     # Обнуление рабочего словаря для данных по проверяемой валюте
    line = val_base[curr]                                         # Запись данных для конкретной валюты для анализа
#     print(line)                                                 # Контрольный вывод анализируемой строки с данными валюты
    
    if result_name == '':                                         # Проверка для выявления 1го шага цикла
        result_name = line['Name']                                # Запись наименования валюты
        result_value = line['Value']                              # Запись значения курса валюты
    elif line['Name'] == 'СДР (специальные права заимствования)': # Выключение из рейтинга неизвестной валюты
        pass                                                      # Пропуск действия
    elif result_value <= (line['Value'] / line['Nominal']):       # Проверка на превышения найденного ранее МАХ значения
        result_name = line['Name']
        result_value = (line['Value'] / line['Nominal'])          # С пересчётом (приведением) цены к единице номинала
        print('Max-value currency now is: ', result_name, ' with value =', result_value, ' rub') # Контрольный вывод рез-та
    else:
        pass

print('==================================')
print('Max-value currency today is: ', result_name, ' with value =', result_value, ' rub')
print('And currency /СДР (специальные права заимствования)/ was not in rang')
print('Process finished')

Max-value currency now is:  Фунт стерлингов Соединенного королевства  with value = 98.3344  rub
Max-value currency today is:  Фунт стерлингов Соединенного королевства  with value = 98.3344  rub
And currency /СДР (специальные права заимствования)/ was not in rang
Process finished


## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [71]:
import requests

class Rate:
    def __init__(self, format_ = 'value', diff='False'):
        self.format = format_
        self.diff = diff
        
        if format_ == 'value':
            self.diff = 'True'
        else:
            self.diff = 'False'
    
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return r.json()['Valute']
    
    def make_format(self, currency):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format_ = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        response = self.exchange_rates()
        
        if currency in response:
            if self.format_ == 'full':
                return response[currency]
            
            if self.format_ == 'value':
                return response[currency]['Value']
        
        return 'Error'
    
    def eur(self, diff):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self, diff):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')

    def AZN(self, diff):
        """Возвращает курс азербайджанского маната на сегодня в формате self.format"""
        return self.make_format('AZN')

r = Rate(format_='full')
r.eur

<bound method Rate.eur of <__main__.Rate object at 0x000001AF33F3D940>>

In [ ]:
from libs.exchange import Rate

class MyRate(Rate):
    def __init__(self):
        super().__init__(format_='full')
        self.diff = 'True'

# mr = MyRate()

 # Задание 3
Напишите класс Designer, который учитывает количество международных премий. Подсказки в коде занятия в разделе “Домашнее задание задача 3”.

## Комментарий по классу Designer такой:
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия. Комментарий про его условия Вика написала выше: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”

In [11]:
class Designer:
    def __init__(self, name, grade, score):
        self.name = name
        self.grade = grade
        self.score = score
    
    def up(self):
        if self.score >= 7:
            self.grade += 1
            self.score = self.score - (7 * (self.score % 7))
            print('Грейд дизайнера ', self.name, ' повышен. Новый грейд: ', self.grade)
            print('Остаток баллов грейда: ', self.score)
        else:
            print('Не достаточно баллов для повышения грейда дизайнера: ', self.score)
            
    def bonus(self):
        self.score += 2
        print('Получена международная премия. Баллов грейдирования: ', self.score)
    
    def print_(self):
        print('Сотрудник {}, грейд {}'.format(self.name, self.grade, self.score))

des1 = Designer(name='Энтони', grade=1, score=4)
des1.print_()
des1.bonus()
des1.print_()
des1.up()
des1.bonus()
des1.print_()
des1.up()
des1.print_()

Сотрудник Энтони, грейд 1
Получена международная премия. Баллов грейдирования:  6
Сотрудник Энтони, грейд 1
Не достаточно баллов для повышения грейда дизайнера:  6
Получена международная премия. Баллов грейдирования:  8
Сотрудник Энтони, грейд 1
Грейд дизайнера  Энтони  повышен. Новый грейд:  2
Остаток баллов грейда:  1
Сотрудник Энтони, грейд 2
